In [ ]:
import tika.tika
from tika import parser
import os
from backports import csv
import codecs
import io
import shutil

In [ ]:
home = os.getenv('HOME')
tika.tika.TikaServerClasspath = home + 'TEAM_10_DSCI550_HW_EXTRACT/Source Code/src/location-ner-model/org/apache/tika/mime'+home+'TEAM_10_DSCI550_HW_EXTRACT/Source Code/src/loccation-ner-model/org/apache/tika/parser/geo'

In [ ]:
input_csv_path = home + 'TEAM_10_DSCI550_HW_EXTRACT/Data/task6_dataset.csv'
output_csv_path = home + 'TEAM_10_DSCI550_HW_EXTRACT/Data/task7_dataset_v1.csv'
temp_dir_path = home + 'TEAM_10_DSCI550_HW_EXTRACT/Data/temp'

In [ ]:
if not os.path.exists(temp_dir_path):
    os.makedirs(temp_dir_path)

with io.open(input_csv_path, 'r', encoding='utf-8') as csvfile, io.open(output_csv_path, 'w', encoding='utf-8') as output_file:
    csvreader = csv.DictReader(csvfile)
    fieldnames = csvreader.fieldnames + ['Location', 'Longitude', 'Latitude']

    csvwriter = csv.DictWriter(output_file, fieldnames=fieldnames)
    csvwriter.writeheader()

    for row in csvreader:
        fields = ['State', 'County', 'Location Details', 'Nearest Town', 'Nearest Road']
        descriptions = ["This location is in the United States, the State of this location is the", "State, the County is the", "County, the location details are described as",
                        "and, the nearest town is", "and the nearest road is"]

        text_to_parse = u' '.join(
            u"{} {}".format(desc, row[field]) for field, desc in zip(fields, descriptions) if row[field].strip()
        )

        temp_geot_file_path = os.path.join(temp_dir_path, 'temp.geot')
        with io.open(temp_geot_file_path, 'w', encoding='utf-8') as temp_file:
            temp_file.write(text_to_parse)

        parsed = parser.from_file(temp_geot_file_path)

        row['Location'] = parsed["metadata"].get('Geographic_NAME', '')
        row['Longitude'] = parsed["metadata"].get('Geographic_LONGITUDE', '')
        row['Latitude'] = parsed["metadata"].get('Geographic_LATITUDE', '')

        csvwriter.writerow(row)
        os.remove(temp_geot_file_path)

shutil.rmtree(temp_dir_path)

In [ ]:
input_csv_path = home + 'TEAM_10_DSCI550_HW_EXTRACT/Data/task7_dataset_v1.csv'
output_csv_path = home + 'TEAM_10_DSCI550_HW_EXTRACT/Data/task7_dataset.csv'

if not os.path.exists(temp_dir_path):
    os.makedirs(temp_dir_path)

with io.open(input_csv_path, 'r', encoding='utf-8') as csvfile, io.open(output_csv_path, 'w', encoding='utf-8') as output_file:
    csvreader = csv.DictReader(csvfile)
    fieldnames = csvreader.fieldnames
    csvwriter = csv.DictWriter(output_file, fieldnames=fieldnames)
    csvwriter.writeheader()

    for row in csvreader:
        if (not row.get('Location') or not row.get('Longitude') or not row.get('Latitude')) and row.get('State'):
            text_to_parse = row['State']

            temp_geot_file_path = os.path.join(temp_dir_path, 'temp.geot')
            with io.open(temp_geot_file_path, 'w', encoding='utf-8') as temp_file:
                temp_file.write(text_to_parse)

            parsed = parser.from_file(temp_geot_file_path)

            if not row.get('Location'):
                row['Location'] = parsed["metadata"].get('Geographic_NAME', '')
            if not row.get('Longitude'):
                row['Longitude'] = parsed["metadata"].get('Geographic_LONGITUDE', '')
            if not row.get('Latitude'):
                row['Latitude'] = parsed["metadata"].get('Geographic_LATITUDE', '')

            os.remove(temp_geot_file_path)

        csvwriter.writerow(row)

shutil.rmtree(temp_dir_path)

In [ ]:
if os.path.exists(input_csv_path):
    os.remove(input_csv_path)